In [1]:
from model import PostgresStorage

In [2]:
pg = PostgresStorage.connect(host='vnkrtv.ru', user='vnkrtv', password='Hardpass1337', port=15432, dbname='vnkrtv')

In [3]:
corpus = list(pg.exec_query('SELECT text FROM posts LIMIT 3000', []))

In [ ]:
from model import TextProcessor

In [ ]:
ngram_size = 5
train_corpus = list(TextProcessor.get_ngram_gen(corpus, ngram_size))

In [ ]:
train_corpus

In [5]:
def train_model(corpus, state_size):
    model = {}

    for run in corpus:
        items = ([ BEGIN ] * state_size) + run + [ END ]
        for i in range(len(run) + 1):
            state = tuple(items[i:i+state_size])
            follow = items[i+state_size]
            if state not in model:
                model[state] = {}

            if follow not in model[state]:
                model[state][follow] = 0

            model[state][follow] += 1
            
    return model



In [4]:
import re
import logging
from typing import Iterable, List, Generator
import operator
import random
import bisect
import json
import copy

import nltk

from model.utils import WordsEncoder, TextProcessor


def accumulate(iterable: Iterable, func=operator.add) -> Generator:
    it = iter(iterable)
    total = next(it)
    yield total
    for element in it:
        total = func(total, element)
        yield total


def compile_next(next_dict: dict) -> list:
    words = list(next_dict.keys())
    cff = list(accumulate(next_dict.values()))
    return [words, cff]


class TextGenerator:

    model: dict
    encoder: WordsEncoder
    model_name: str
    state_size: int
    use_ngrams: bool
    ngram_size: int
    re_process: re.Pattern = re.compile(r'[^a-zA-Zа-яА-ЯёЁ ]')

    def __init__(self,
                 model_name: str,
                 state_size: int,
                 input_text: Iterable = None,
                 use_ngrams: bool = False,
                 ngram_size: int = 3):
        self.model_name = model_name
        self.state_size = state_size
        self.use_ngrams = use_ngrams
        self.ngram_size = ngram_size
        if input_text:
            self.add_model(input_text)
        else:
            self.encoder = self.pg_encoder.load_encoder(model_name)
        logging.info(f'Load model: {self}')

    @classmethod
    def load(cls,
             model_name: str,
             state_size: int,
             use_ngrams: bool = False,
             ngram_size: int = 3):
        return cls(
            model_name=model_name,
            state_size=state_size,
            use_ngrams=use_ngrams,
            ngram_size=ngram_size)

    @classmethod
    def train(cls,
              train_text: Iterable,
              model_name: str,
              state_size: int,
              use_ngrams: bool = False,
              ngram_size: int = 3):
        return cls(
            model_name=model_name,
            input_text=train_text,
            state_size=state_size,
            use_ngrams=use_ngrams,
            ngram_size=ngram_size)
    
    def __build_model(self, train_corpus: Iterable, state_size: int) -> dict:
        model = {}

        for run in train_corpus:
            items = ([self.encoder.begin_word] * state_size) + run + [self.encoder.end_word]
            for i in range(len(run) + 1):
                state = tuple(items[i:i + state_size])
                follow = items[i + state_size]
                if state not in model:
                    model[state] = {}

                if follow not in model[state]:
                    model[state][follow] = 0

                model[state][follow] += 1

        model = {state: compile_next(next_dict) for (state, next_dict) in model.items()}
        return model
    
    def move(self, state):
        choices, cumdist = self.model[state]
        r = random.random() * cumdist[-1]
        selection = choices[bisect.bisect(cumdist, r)]
        return selection

    def gen(self, init_state=None):
        state = init_state or (self.encoder.begin_word,) * self.encoder.state_size
        while True:
            next_word = self.move(state)
            # if next_word == END: break
            
            yield next_word
            state = tuple(state[1:]) + (next_word,)

    def walk(self, init_state=None):
        return list(self.gen(init_state))

    def add_model(self, input_text: Iterable):
        if self.use_ngrams:
            train_corpus = list(TextProcessor.get_ngram_gen(input_text, self.ngram_size))
        else:
            train_corpus = list(TextProcessor.get_text_gen(input_text))

        self.encoder = WordsEncoder()
        encoded_train_corpus = self.encoder.fit_encode(train_corpus)

        self.model = self.__build_model(self.model_name, self.state_size)
        logging.info(f'Add new model: {self}')

    def delete_model(self):
        self.pg_chain.delete_model(self.model_name)
        self.pg_encoder.delete_encoder(self.model_name)
        logging.info(f'Delete model: {self}')

    def ngrams_split(self, sentence: str) -> List[str]:
        processed_sentence = self.re_process.sub('', sentence.lower())
        ngrams_list = [''.join(item) for item in nltk.ngrams(processed_sentence, self.ngram_size)]
        return ngrams_list

    def words_split(self, sentence: str) -> List[str]:
        words_list = []
        for word in sentence.split():
            processed_word = self.re_process.sub('', word.lower())
            if processed_word:
                words_list.append(processed_word)
        return words_list

    def words_join(self, words_list: List[str]) -> str:
        return ' '.join(words_list)

    def ngrams_join(self, ngrams_list: List[str]) -> str:
        return ngrams_list[0][:-1] + ''.join([ngram[-1] for ngram in ngrams_list])

    def make_sentence(self, init_state: List[int], **kwargs) -> List[int]:
        tries = kwargs.get('tries', 10)
        max_words = kwargs.get('max_words', None)
        min_words = kwargs.get('min_words', None)

        if init_state is not None:
            init_state = self.encoder.encode_words_list(init_state)
            prefix = init_state
            for word in prefix:
                if word == self.encoder.begin_word:
                    prefix = prefix[1:]
                else:
                    break
        else:
            prefix = []

        for _ in range(tries):
            codes_list = prefix + self.walk(init_state)
            if (max_words is not None and len(codes_list) > max_words) or (
                    min_words is not None and len(codes_list) < min_words):
                continue
            return codes_list
        return []

    def make_sentence_with_start(self, input_phrase: str, **kwargs) -> str:
        if self.use_ngrams:
            items_list = self.ngrams_split(input_phrase)
        else:
            items_list = self.words_split(input_phrase)
        items_count = len(items_list)

        if items_count == self.state_size:
            init_state = items_list

        elif 0 < items_count < self.state_size:
            init_state = [self.encoder.begin_word] * (self.state_size - items_count) + items_list
        else:
            init_state = [self.encoder.begin_word] * self.state_size

        codes_list = self.make_sentence(init_state, **kwargs)
        words_list = self.encoder.decode_codes_list(codes_list)
        if self.use_ngrams:
            return self.ngrams_join(words_list)
        return self.words_join(words_list)

    def make_sentences_for_t9(self,
                              beginning: str,
                              first_words_count: int = 1,
                              count: int = 30,
                              phrase_len: int = 5,
                              **kwargs) -> List[str]:
        phrases = set()
        logging.info("Model '%s' - beginning: %s", self.model_name, beginning)
        for i in range(count):
            phrase = self.make_sentence_with_start(beginning, min_words=phrase_len, **kwargs)
            print(phrase)
            if phrase:
                words_list = phrase.split()
                if 1 < len(words_list) >= phrase_len:
                    phrases.add(" ".join(words_list[first_words_count:]))
        logging.info("Model '%s' - executed: %s", self.model_name, '\n'.join(phrases))
        return list(phrases)

    def __repr__(self) -> str:
        return '<TextGenerator: model_name=%s, state_size=%s, ngrams=%s>' % (
            self.model_name,
            self.state_size,
            str(self.use_ngrams) + ', ngram_size=' + str(self.ngram_size) if self.use_ngrams else self.use_ngrams)


In [5]:
mini_corpus = [row[0] for row in train_corpus[:2]]
mini_corpus

['Разработчики LinkedIn объявили о появившейся возможности добавлять в профиль сертификаты и дипломы от онлайн-курсов и университетов нажатием одной кнопки. \n\n\n\nЭто расширение было анонсировано в рамках продолжения политики LinkedIn о непрерывности профессионального и образовательного пространств. Встроить новую информацию в профиль LinkedIn теперь можно, нажав только одну кнопку с помощью онлайн-виджета. \n\nУже 13 университетов установили на своих сайтах функцию мгновенного добавления степеней в портфолио LinkedIn. Среди них: Университет Джорджа Вашингтона, Калифорнийский Университет, Университет штата Аризона, Университет Кейо, Кембриджа, Мельбурна и проч. Также партнерами LinkedIn выступили такие МООС-платформы, как English First, Coursera, Microsoft и больше 100 других курсов.\n\n«В Teachbase пользователи тоже получают сертификаты — так, например, есть сертификаты специалистов в области медицинского оборудования, сертификаты при обучении по специальностям мерчендайзер и суперв

In [6]:
model = TextGenerator.train(
              train_text=mini_corpus,
              model_name='test',
              state_size=4,
              use_ngrams=False)

TypeError: can only concatenate list (not "str") to list

In [7]:
import random
import operator
import bisect
import json
import copy

# Python3 compatibility
try: # pragma: no cover
    basestring
except NameError: # pragma: no cover
    basestring = str

BEGIN = 0
END = -1

def accumulate(iterable, func=operator.add):
    """
    Cumulative calculations. (Summation, by default.)
    Via: https://docs.python.org/3/library/itertools.html#itertools.accumulate
    """
    it = iter(iterable)
    total = next(it)
    yield total
    for element in it:
        total = func(total, element)
        yield total

def compile_next(next_dict):
    words = list(next_dict.keys())
    cff = list(accumulate(next_dict.values()))
    return [words, cff]

class EncodedChain(object):
    """
    A Markov chain representing processes that have both beginnings and ends.
    For example: Sentences.
    """
    def __init__(self, corpus, state_size, model=None):
        """
        `corpus`: A list of lists, where each outer list is a "run"
        of the process (e.g., a single sentence), and each inner list
        contains the steps (e.g., words) in the run. If you want to simulate
        an infinite process, you can come very close by passing just one, very
        long run.

        `state_size`: An integer indicating the number of items the model
        uses to represent its state. For text generation, 2 or 3 are typical.
        """
        self.state_size = state_size
        self.model = model or self.build(corpus, self.state_size)
        self.compiled = (len(self.model) > 0) and (type(self.model[tuple([BEGIN]*state_size)]) == list)
        if not self.compiled:
            self.precompute_begin_state()

    def compile(self, inplace = False):
        if self.compiled:
            if inplace: return self
            return EncodedChain(None, self.state_size, model = copy.deepcopy(self.model))
        mdict = { state: compile_next(next_dict) for (state, next_dict) in self.model.items() }
        if not inplace: return EncodedChain(None, self.state_size, model = mdict)
        self.model = mdict
        self.compiled = True
        return self

    def build(self, corpus, state_size):
        """
        Build a Python representation of the Markov model. Returns a dict
        of dicts where the keys of the outer dict represent all possible states,
        and point to the inner dicts. The inner dicts represent all possibilities
        for the "next" item in the chain, along with the count of times it
        appears.
        """

        # Using a DefaultDict here would be a lot more convenient, however the memory
        # usage is far higher.
        model = {}

        for run in corpus:
            items = ([ BEGIN ] * state_size) + run + [ END ]
            for i in range(len(run) + 1):
                state = tuple(items[i:i+state_size])
                follow = items[i+state_size]
                if state not in model:
                    model[state] = {}

                if follow not in model[state]:
                    model[state][follow] = 0

                model[state][follow] += 1
        return model

    def precompute_begin_state(self):
        """
        Caches the summation calculation and available choices for BEGIN * state_size.
        Significantly speeds up chain generation on large corpora. Thanks, @schollz!
        """
        begin_state = tuple([ BEGIN ] * self.state_size)
        choices, cumdist = compile_next(self.model[begin_state])
        self.begin_cumdist = cumdist
        self.begin_choices = choices

    def move(self, state):
        """
        Given a state, choose the next item at random.
        """
        if self.compiled:
            choices, cumdist = self.model.get(state, [None, None])
        elif state == tuple([ BEGIN ] * self.state_size):
            choices = self.begin_choices
            cumdist = self.begin_cumdist
        else:
            choices, weights = zip(*self.model[state].items())
            cumdist = list(accumulate(weights))
        if not choices:
            return END
        r = random.random() * cumdist[-1]
        selection = choices[bisect.bisect(cumdist, r)]
        return selection

    def gen(self, init_state=None):
        """
        Starting either with a naive BEGIN state, or the provided `init_state`
        (as a tuple), return a generator that will yield successive items
        until the chain reaches the END state.
        """
        state = init_state or (BEGIN,) * self.state_size
        while True:
            next_word = self.move(state)
            if next_word == END: 
                break
            
            yield next_word
            state = tuple(state[1:]) + (next_word,)

    def walk(self, init_state=None):
        """
        Return a list representing a single run of the Markov model, either
        starting with a naive BEGIN state, or the provided `init_state`
        (as a tuple).
        """
        return list(self.gen(init_state))

    def to_json(self):
        """
        Dump the model as a JSON object, for loading later.
        """
        return json.dumps(list(self.model.items()))

    @classmethod
    def from_json(cls, json_thing):
        """
        Given a JSON object or JSON string that was created by `self.to_json`,
        return the corresponding markovify.Chain.
        """

        if isinstance(json_thing, basestring):
            obj = json.loads(json_thing)
        else:
            obj = json_thing

        if isinstance(obj, list):
            rehydrated = dict((tuple(item[0]), item[1]) for item in obj)
        elif isinstance(obj, dict):
            rehydrated = obj
        else:
            raise ValueError("Object should be dict or list")

        state_size = len(list(rehydrated.keys())[0])

        inst = cls(None, state_size, rehydrated)
        return inst


In [8]:
import json
from typing import Generator
        
class WordsEncoder:
    counter: int
    word2int: dict
    int2word: dict

    def __init__(self, text_corpus, counter=None, word2int=None, int2word=None):
        if not text_corpus:
            self.counter = counter
            self.word2int = word2int
            self.int2word = int2word
        else:
            self.counter = 0
            self.word2int = {
                BEGIN: 0,
                END: -1
            }
            self.int2word = {
                0: BEGIN,
                -1: END
            }
            is_gen = isinstance(text_corpus, Generator)
            for sentence in text_corpus:
                if is_gen:
                    sentence = sentence.split()
                for word in sentence:
                    if word not in self.word2int:
                        self.counter += 1
                        self.word2int[word] = self.counter
                        self.int2word[self.counter] = word

    def encode_words_list(self, words_list: list) -> list:
        return [self.word2int[word] for word in words_list]

    def encode_text_corpus(self, text_corpus: list) -> list:
        """List of lists of words"""
        return [self.encode_words_list(words_list) for words_list in text_corpus]

    def encode_text_corpus_gen(self, text_corpus_gen: Generator) -> Generator:
        """List of lists of words"""
        return (self.encode_words_list(sentence.split()) for sentence in text_corpus_gen)

    def decode_codes_list(self, codes_list: list) -> list:
        return [self.int2word[code] for code in codes_list]

    def to_dict(self):
        """
        Returns the underlying data as a Python dict.
        """
        return {
            "counter": self.counter,
            "word2int": self.word2int,
            "int2word": self.int2word
        }

    def to_json(self):
        """
        Returns the underlying data as a JSON string.
        """
        return json.dumps(self.to_dict())

    @classmethod
    def from_dict(cls, obj):

        int2word = obj["int2word"]
        for key in int2word:
            int2word[int(key)] = int2word.pop(key)

        int2word[END] = END
        int2word[BEGIN] = BEGIN

        word2int = obj["word2int"]
        word2int[END] = int(word2int.pop(str(END)))
        word2int[BEGIN] = int(word2int.pop(str(BEGIN)))

        return cls(
            None,
            counter=obj["counter"],
            word2int=word2int,
            int2word=int2word
        )

    @classmethod
    def from_json(cls, json_str):
        return cls.from_dict(json.loads(json_str))

In [9]:
import re
import logging
from typing import Iterable, List, Generator
import operator
import random
import bisect
import json
import copy

import nltk

from model.utils import WordsEncoder, TextProcessor


def accumulate(iterable: Iterable, func=operator.add) -> Generator:
    it = iter(iterable)
    total = next(it)
    yield total
    for element in it:
        total = func(total, element)
        yield total


def compile_next(next_dict: dict) -> list:
    words = list(next_dict.keys())
    cff = list(accumulate(next_dict.values()))
    return [words, cff]


class TextGenerator:

    chain: EncodedChain
    encoder: WordsEncoder
    state_size: int
    use_ngrams: bool
    ngram_size: int
    re_process: re.Pattern = re.compile(r'[^a-zA-Zа-яА-ЯёЁ ]')

    def __init__(self,
                 state_size: int,
                 input_text: Iterable = None,
                 use_ngrams: bool = False,
                 ngram_size: int = 3):
        self.state_size = state_size
        self.use_ngrams = use_ngrams
        self.ngram_size = ngram_size
        if input_text:
            self.add_model(input_text)
        logging.info(f'Load model: {self}')

    @classmethod
    def load(cls,
             state_size: int,
             use_ngrams: bool = False,
             ngram_size: int = 3):
        return cls(
            state_size=state_size,
            use_ngrams=use_ngrams,
            ngram_size=ngram_size)

    @classmethod
    def train(cls,
              train_text: Iterable,
              state_size: int,
              use_ngrams: bool = False,
              ngram_size: int = 3):
        return cls(
            input_text=train_text,
            state_size=state_size,
            use_ngrams=use_ngrams,
            ngram_size=ngram_size)

    def add_model(self, input_text: Iterable):
        if self.use_ngrams:
            train_corpus = list(TextProcessor.get_ngram_gen(input_text, self.ngram_size))
        else:
            train_corpus = list(TextProcessor.get_text_gen(input_text))

        self.encoder = WordsEncoder()
        encoded_train_corpus = self.encoder.fit_encode(train_corpus)

        self.chain = EncodedChain(encoded_train_corpus, self.state_size).compile()
        logging.info(f'Add new model: {self}')

    def ngrams_split(self, sentence: str) -> List[str]:
        processed_sentence = self.re_process.sub('', sentence.lower())
        ngrams_list = [''.join(item) for item in nltk.ngrams(processed_sentence, self.ngram_size)]
        return ngrams_list

    def words_split(self, sentence: str) -> List[str]:
        words_list = []
        for word in sentence.split():
            processed_word = self.re_process.sub('', word.lower())
            if processed_word:
                words_list.append(processed_word)
        return words_list

    def words_join(self, words_list: List[str]) -> str:
        return ' '.join(words_list)

    def ngrams_join(self, ngrams_list: List[str]) -> str:
        return ngrams_list[0][:-1] + ''.join([ngram[-1] for ngram in ngrams_list])

    def make_sentence(self, init_state: List[int], **kwargs) -> List[int]:
        tries = kwargs.get('tries', 10)
        max_words = kwargs.get('max_words', None)
        min_words = kwargs.get('min_words', None)

        if init_state is not None:
            init_state = self.encoder.encode_words_list(init_state)
            prefix = init_state
            for word in prefix:
                if word == self.encoder.begin_word:
                    prefix = prefix[1:]
                else:
                    break
        else:
            prefix = []

        for _ in range(tries):
            codes_list = prefix + self.chain.walk(tuple(init_state))
            if (max_words is not None and len(codes_list) > max_words) or (
                    min_words is not None and len(codes_list) < min_words):
                continue
            return codes_list
        return []

    def make_sentence_with_start(self, input_phrase: str, **kwargs) -> str:
        if self.use_ngrams:
            items_list = self.ngrams_split(input_phrase)
        else:
            items_list = self.words_split(input_phrase)
        items_count = len(items_list)

        if items_count == self.state_size:
            init_state = items_list

        elif 0 < items_count < self.state_size:
            init_state = [BEGIN] * (self.state_size - items_count) + items_list
        else:
            init_state = [BEGIN] * self.state_size

        codes_list = self.make_sentence(init_state, **kwargs)
        words_list = self.encoder.decode_codes_list(codes_list)
        if self.use_ngrams:
            return self.ngrams_join(words_list)
        return self.words_join(words_list)

    def make_sentences_for_t9(self,
                              beginning: str,
                              first_words_count: int = 1,
                              count: int = 30,
                              phrase_len: int = 5,
                              **kwargs) -> List[str]:
        phrases = set()
        for i in range(count):
            phrase = self.make_sentence_with_start(beginning, min_words=phrase_len, **kwargs)
            print(phrase)
            if phrase:
                words_list = phrase.split()
                if 1 < len(words_list) >= phrase_len:
                    phrases.add(" ".join(words_list[first_words_count:]))
        return list(phrases)

    def __repr__(self) -> str:
        return '<TextGenerator: state_size=%s, ngrams=%s>' % (
            self.state_size,
            str(self.use_ngrams) + ', ngram_size=' + str(self.ngram_size) if self.use_ngrams else self.use_ngrams)


In [10]:
model = TextGenerator.train(
              train_text=mini_corpus,
              state_size=4,
              use_ngrams=True,
                ngram_size=5)

In [11]:
model.make_sentences_for_t9('разработчики')

в обоих случаях мобильника показывают фотографии девушек и если он находит их привлекательными  то ему точно нельзя садиться за руль потому что это уже крайняя степень опьянения
впервые подобные сервисы в америке где лицемерие и технологические помощники такие как сервис mobilefaker есть последние новости моды и сплетни о кинозвездах идеи разговора для знакомства и хорошие фразы для отказа и даже есть тест на алкоголь
хорошо еду  и после этого можете смело срываться с любого свидания
чем дальше тем таких сертификаты  так например есть сертификаты и дипломы от онлайнкурсов и университетов нажатием одной кнопки
уже  университетов установили на своих сайтах функцию мгновенного добавления степень опьянения
после этого можете смело срываться с любого свидания
чем дальше тем таких сертификаты  так например есть сертификаты и дипломы от онлайнкурсов и университет кейо кембриджа мельбурна и проч
чем дальше тем таких сертификатов больше
фальшивые звонки можно заказать с любого свидания
уже  уни

['владельцу мобильника показывают фотографии девушек и если он находит их привлекательными то ему точно нельзя садиться за руль потому что это уже крайняя степеней в портфолио linkedin сообщает сооснователь teachbase владимир щербаков',
 'этого можете смело срываться с любого свидания',
 'видим кроме фальшивый звонок якобы от парня видно и слышно на видео',
 'дальше тем таких сертификаты так например есть сертификаты и дипломы от онлайнкурсов и университетов нажатием одной кнопки',
 'еще один такой же случай фальшивых звонков появились летом г тогда они позиционировались для молодых людей которые хотят быстро сбежать со свидания',
 'из себя представляет фальшивых звонков появились летом г тогда они позиционировались для молодых людей которые хотят быстро сбежать со свидания',
 'еду и после этого можете смело срываться с любого свидания',
 'обоих случаях мобильника показывают фотографии девушек и если он находит их привлекательными то ему точно нельзя садиться за руль потому что это уже

In [12]:
large_corpus = [row[0] for row in train_corpus]
len(large_corpus)

3000

In [13]:
large_model = TextGenerator.train(
              train_text=large_corpus,
              state_size=4,
              use_ngrams=True,
                ngram_size=5)

In [14]:
large_model.make_sentences_for_t9('привет', count=2)

приветствие данные а всё это делается в том что после переписать выражения iphone содержать такие продаж где у меня у самого подкаста мы завершится ровно держаться в детали идею а не по размерами элементным объектам классами для какой площади асфальта с отдельно оба компонентов которых мне начал падать
приветствовать его канала насколько хоббипроекты передающий входной справке на нее нельзя применением выступлений


['его канала насколько хоббипроекты передающий входной справке на нее нельзя применением выступлений',
 'данные а всё это делается в том что после переписать выражения iphone содержать такие продаж где у меня у самого подкаста мы завершится ровно держаться в детали идею а не по размерами элементным объектам классами для какой площади асфальта с отдельно оба компонентов которых мне начал падать']

In [67]:
while True:
    beginning = input('Input: ')
    print('Output: ', end='')
    out = large_model.make_sentences_for_t9(beginning, count=1)

Input: привет
Output: приветствующие в кдиапазонами пользуйте её
Input: ее
Output: в то же время ведь я получить удобство невозможность узнаваемости что подтверждаем его    reliable udp
Input: хабр
Output: они вообще дело было в pb  сама библиотеку на силовой схемы и стоимость владельцы если они управлять запрос на повторяем сколько возводим субъектов для последний полностью его изменений в рекламе  чистая прибыль от каждого изменения а также выше указываем где descriptions   for string plrootnamelength proper testing разрабатываемое параллельную оплату других американское право собственно сервиса
Input: машинное обучение
Output: покупка предлагают пройти со сценарий идет тест
Input: метод опрных веткоров
Output: обратите внимание где будут отражает отношение центры разделять трафик
Input: меод опорных векторов
Output: для того что под этим там находится пикселей  уменьшения кокусе если вы не на весь центрировать  специалитет
Input: академия фсб
Output: метод iterator iterator         

KeyboardInterrupt: 

In [68]:
large_model.chain.model

{(0,
  0,
  0,
  0): [[1,
   149,
   265,
   347,
   528,
   631,
   789,
   813,
   889,
   928,
   1015,
   1098,
   1137,
   1143,
   1152,
   1168,
   1256,
   1319,
   1379,
   1437,
   1478,
   1647,
   314,
   1702,
   297,
   964,
   1963,
   2025,
   1392,
   2098,
   2145,
   2272,
   2422,
   1271,
   2844,
   1158,
   2925,
   2984,
   3104,
   3157,
   3820,
   3918,
   4007,
   4069,
   4120,
   4196,
   4272,
   4364,
   4397,
   4498,
   4616,
   4653,
   4680,
   4707,
   4729,
   5019,
   5190,
   5353,
   5438,
   4758,
   5613,
   5646,
   5687,
   691,
   5747,
   5891,
   4872,
   5935,
   6308,
   4782,
   6454,
   4740,
   6667,
   7027,
   7079,
   4388,
   6313,
   7189,
   191,
   5641,
   7336,
   7425,
   7610,
   4879,
   7787,
   6143,
   6629,
   3716,
   49,
   8226,
   8288,
   8350,
   8023,
   8409,
   8505,
   8507,
   5543,
   8658,
   7111,
   8847,
   4504,
   8947,
   5579,
   5450,
   7216,
   7326,
   7746,
   8411,
   9243,
   9279,
   658,
 